In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import random
import string
import tensorflow as tf
from tensorflow.python.ops.rnn_cell import _linear
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import collections
import matplotlib.pyplot as plt
import codecs
import time
import os
import gc
from six.moves import cPickle as pickle

from plot_module import text_plot
from plot_module import structure_vocabulary_plots
from plot_module import ComparePlots

from model_module import maybe_download
from model_module import read_data
from model_module import check_not_one_byte
from model_module import id2char
from model_module import char2id
from model_module import BatchGenerator
from model_module import characters
from model_module import batches2string
from model_module import logprob
from model_module import sample_distribution
from model_module import MODEL

In [5]:
if not os.path.exists('enwik8_filtered'):
    if not os.path.exists('enwik8'):
        filename = maybe_download('enwik8.zip', 36445475)
    full_text = read_data(filename)
    new_text = u""
    new_text_list = list()
    for i in range(len(full_text)):
        if (i+1) % 10000000 == 0:
            print("%s characters are filtered" % i)
        if ord(full_text[i]) < 256:
            new_text_list.append(full_text[i])
    text = new_text.join(new_text_list)
    del new_text_list
    del new_text
    del full_text

    (not_one_byte_counter, min_character_order_index, max_character_order_index, number_of_characters, present_characters_indices) = check_not_one_byte(text)

    print("number of not one byte characters: ", not_one_byte_counter) 
    print("min order index: ", min_character_order_index)
    print("max order index: ", max_character_order_index)
    print("total number of characters: ", number_of_characters)
    
    f = open('enwik8_filtered', 'w')
    f.write(text.encode('utf8'))
    f.close()
    
else:
    f = open('enwik8_filtered', 'r')
    text = f.read().decode('utf8')
    f.close() 
    (not_one_byte_counter, min_character_order_index, max_character_order_index, number_of_characters, present_characters_indices) = check_not_one_byte(text)

    print("number of not one byte characters: ", not_one_byte_counter) 
    print("min order index: ", min_character_order_index)
    print("max order index: ", max_character_order_index)
    print("total number of characters: ", number_of_characters)    

number of not one byte characters:  0
min order index:  9
max order index:  255
total number of characters:  196


In [6]:
#different
offset = 20000
valid_size = 22500
valid_text = text[offset:offset+valid_size]
train_text = text[offset+valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99337500 side of the leftist milieu. It often focuses on the individual r
22500 ture in Mutual Aid: A Factor of Evolution (1897). Subsequent ana


In [7]:
vocabulary_size = number_of_characters
vocabulary = list()
characters_positions_in_vocabulary = list()

character_position_in_vocabulary = 0
for i in range(256):
    if present_characters_indices[i]:
        vocabulary.append(unichr(i))
        characters_positions_in_vocabulary.append(character_position_in_vocabulary)
        character_position_in_vocabulary += 1
    else:
        characters_positions_in_vocabulary.append(-1)


string_vocabulary = u""
for i in range(vocabulary_size):
    string_vocabulary += vocabulary[i]
print("Vocabulary: ", string_vocabulary)
print("char2id(u'a') = %s,  char2id(u'z') = %s,  char2id(u' ') = %s" % (char2id(u'a', characters_positions_in_vocabulary),
                                                                        char2id(u'z', characters_positions_in_vocabulary),
                                                                        char2id(u' ', characters_positions_in_vocabulary)))
print("id2char(78) = %s,  id2char(156) = %s,  id2char(140) = %s" % (id2char(78,
                                                                            vocabulary),
                                                                    id2char(156,
                                                                            vocabulary),
                                                                    id2char(140,
                                                                            vocabulary)))

Vocabulary:  	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿ
char2id(u'a') = 67,  char2id(u'z') = 92,  char2id(u' ') = 2
id2char(78) = l,  id2char(156) = Ø,  id2char(140) = È


In [8]:
batch_size_test=64
num_unrollings_test=10

train_batches_test = BatchGenerator(train_text,
                                    batch_size_test,
                                    vocabulary_size,
                                    characters_positions_in_vocabulary,
                                    num_unrollings_test)
valid_batches_test = BatchGenerator(valid_text,
                                    1,
                                    vocabulary_size,
                                    characters_positions_in_vocabulary,
                                    1)

print(batches2string(train_batches_test.next(), vocabulary))
print(batches2string(train_batches_test.next(), vocabulary))
print(batches2string(valid_batches_test.next(), vocabulary))
print(batches2string(valid_batches_test.next(), vocabulary))

[u'side of the', u'though the ', u'; \n|colspan', u'ustrian.asp', u'e most cons', u' Gaul, whic', u'h can be pr', u'pitome of t', u' first team', u'Roussimoff ', u'col. In Feb', u's]] - [[Uni', u'g oven, dis', u' exposed to', u' wrote the ', u'ird.  He co', u'a]].  The O', u'nd Lower Lo', u'In another ', u'or sensitiv', u'ents used b', u'e Chile]] i', u"achelor's d", u'till employ', u'flexibility', u'nects all c', u'variety of ', u' [[Cheshire', u' sleek, str', u' have led t', u'ongside man', u'eature.\n\n==', u'st and cent', u'oblems incl', u'aunched his', u'310)\n\n*&quo', u'character n', u'ward-winnin', u'ularly elec', u'expense of ', u", '''Eyes W", u';Hesperus i', u'sletter was', u'uction comp', u's. The conc', u'ow]] for mo', u'l football ', u' [[Hayek So', u'ed [[transl', u'amp>2006-03', u'ears, even ', u' plates for', u'[post punk]', u' equal righ', u'ng with onl', u'his English', u'lling the r', u's of Parlia', u' England.  ', u're being ma', u'endered in ', u'okomo]]\n***', u

In [9]:
class adaptive_sum_with_memory(MODEL):
    def layer(self, 
              inp_t,
              state_t_minus_1,
              memory_t_minus_1):
        X_t = tf.concat(1, [inp_t,
                            state_t_minus_1,
                            memory_t_minus_1])
        RES = tf.matmul(X_t, self.Matrix) + self.Bias
        state_t = tf.tanh(RES)
        return state_t
    
    def swap(self, output, i):
        return [tf.slice(output, [i, 0], [1, self._num_nodes[0]]), tf.constant(0., shape=[1, 1])]
    
    def skip_swap(self, memory, i, current_trigger):
        return [tf.slice(memory, [i, 0], [1, self._num_nodes[0]]), tf.slice(current_trigger, [i, 0], [1, 1])]

    
    def iteration(self, inp, state, current_trigger):
        output = self.layer(inp,
                            state[0],
                            state[1])
        trigger = tf.sigmoid(tf.matmul(tf.concat(1, [inp, output, state[1]]), self.trigger_matrix) + self.trigger_bias)
        current_trigger = current_trigger + trigger
        trigger_for_plot = current_trigger
        memory_list = list()
        trigger_list = list()
        current_batch_size = trigger.get_shape().as_list()[0]
        ones = tf.ones([current_batch_size])
        to_swap = tf.to_float(tf.greater(tf.reshape(current_trigger, [-1]), self.thresh)) 
        memory = tf.transpose( tf.transpose(trigger)*tf.transpose(output)*to_swap + tf.transpose(state[1]) * (ones - to_swap))
        current_trigger = tf.transpose( tf.transpose(current_trigger) * (ones - to_swap))
        return output, [output, memory], trigger, current_trigger, trigger_for_plot
    
    def __init__(self,
                 batch_size,
                 vocabulary,
                 characters_positions_in_vocabulary,
                 num_unrollings,
                 num_layers,
                 num_nodes,
                 init_bias,
                 threshhold,    #{'fixed': True/False, 'min':  , 'max':  ,'epochs':  }
                 train_text,
                 valid_text,
                 mean=0.,
                 stddev='default',
                 shift=0.,
                 init_learning_rate=1.):
        self._results = list()
        self._batch_size = batch_size
        self._vocabulary = list(vocabulary)
        self._vocabulary_size = len(vocabulary)
        self._characters_positions_in_vocabulary = characters_positions_in_vocabulary
        self._num_unrollings = num_unrollings
        self._num_layers = num_layers
        self._num_nodes = list(num_nodes)
        self._init_bias = init_bias
        self._threshold = dict(threshold)
        self._train_text = train_text
        self._valid_text = valid_text
        self._valid_size = len(valid_text)
        
        
        self._mean = mean
        
        self._stddev = list()
        if stddev == 'default':
            self._stddev = 1.0 * np.sqrt(1./(num_nodes[0] + vocabulary_size))
        else:
            self._stddev = stddev
            
        self._shift = shift
        self._init_learning_rate = init_learning_rate
        
        self._indices = {"batch_size": 0,
                         "num_unrollings": 1,
                         "num_layers": 2,
                         "num_nodes": 3,
                         "half_life": 4,
                         "decay": 5,
                         "num_steps": 6,
                         "averaging_number": 7,
                         "init_bias": 8,
                         "threshold": 9,
                         "memory_fine":10,
                         "init_mean": 11,
                         "init_stddev": 12,
                         "init_shift": 13,
                         "init_learning_rate": 14,
                         "type": 15}
        self._graph = tf.Graph()
        
        self._last_num_steps = 0
        with self._graph.as_default(): 
            with self._graph.device('/gpu:0'): 
                self.Matrix = tf.Variable(tf.truncated_normal([self._vocabulary_size + 2*self._num_nodes[0],
                                                               self._num_nodes[0]],
                                                              mean=self._mean, stddev=self._stddev))
                self.Bias = tf.Variable([self._shift for _ in range(self._num_nodes[0])])

                # classifier 
                weights = tf.Variable(tf.truncated_normal([self._num_nodes[-1], self._vocabulary_size], stddev = 0.1))
                bias = tf.Variable(tf.zeros([self._vocabulary_size]))
                
                self.trigger_matrix = tf.Variable(tf.truncated_normal([self._vocabulary_size + 2*self._num_nodes[0], 1], stddev = 0.1))
                self.trigger_bias = tf.Variable([self._init_bias])
                
                """PLACEHOLDERS train data"""
                self._train_data = list()
                for _ in range(self._num_unrollings + 1):
                    self._train_data.append(
                        tf.placeholder(tf.float32, shape=[self._batch_size, self._vocabulary_size]))
                train_inputs = self._train_data[: self._num_unrollings]
                train_labels = self._train_data[1:]  # labels are inputs shifted by one time step.
                # Unrolled LSTM loop.

                saved_state = [tf.Variable(tf.zeros([self._batch_size, self._num_nodes[0]]), trainable=False),
                               tf.Variable(tf.zeros([self._batch_size, self._num_nodes[0]]), trainable=False)]
                
                """TRIGGER"""
                saved_trigger = tf.Variable(tf.zeros([self._batch_size, 1]))
                """global step"""
                self._global_step = tf.Variable(0)
                """self.thresh"""
                if self._threshold['fixed']:
                    self.thresh = tf.constant(self._threshold['min'])
                else:
                    thresh_range = self._threshold['max'] - self._threshold['min']
                    self.thresh = tf.minimum(tf.constant(self._threshold['min']) + tf.to_float(self._global_step) / self._threshold['epochs'] * tf.constant(thresh_range), tf.constant(self._threshold['max']))
                outputs = list()
                triggers_for_letters =  list()
                state = saved_state
                current_trigger = saved_trigger
                for inp in train_inputs:
                    output, state, trigger_for_letter, current_trigger, _ = self.iteration(inp, state, current_trigger)
                    outputs.append(output)
                    triggers_for_letters.append(trigger_for_letter)

                save_list = list()
                save_list.append(saved_state[0].assign(state[0]))
                save_list.append(saved_state[1].assign(state[1]))
                save_list.append(saved_trigger.assign(current_trigger))
                
                """skip operation"""
                self._skip_operation = tf.group(*save_list)
                
                self.memory_fine = tf.placeholder(tf.float32)
                with tf.control_dependencies(save_list):
                        # Classifier.
                    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), weights, bias)
                    """loss"""
                    self._loss = tf.reduce_mean(
                        tf.nn.softmax_cross_entropy_with_logits(
                        logits, tf.concat(0, train_labels)))
                    fact_loss = tf.reduce_mean(
                        tf.nn.softmax_cross_entropy_with_logits(
                        logits, tf.concat(0, train_labels))) + self.memory_fine * tf.reduce_sum(current_trigger)
                # Optimizer.
                
                """PLACEHOLDERS half life and decay"""
                self._half_life = tf.placeholder(tf.int32)
                self._decay = tf.placeholder(tf.float32)
                """learning rate"""
                self._learning_rate = tf.train.exponential_decay(0.5,
                                                                 self._global_step,
                                                                 self._half_life,
                                                                 self._decay,
                                                                 staircase=True)
                optimizer = tf.train.GradientDescentOptimizer(self._learning_rate)
                gradients, v = zip(*optimizer.compute_gradients(fact_loss))
                gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
                gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
                """optimizer"""
                self._optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=self._global_step)
                """train prediction"""
                self._train_prediction = tf.nn.softmax(logits)

                # Sampling and validation eval: batch 1, no unrolling.
                saved_sample_state = list()
                saved_sample_state.append(tf.Variable(tf.zeros([1, self._num_nodes[0]]), trainable=False)) 
                saved_sample_state.append(tf.Variable(tf.zeros([1, self._num_nodes[0]]), trainable=False)) 
                saved_sample_trigger = tf.Variable(tf.zeros([1, 1]))
                """PLACEHOLDER sample input"""
                self._sample_input = tf.placeholder(tf.float32, shape=[1, self._vocabulary_size])

                reset_list = list()
                reset_list.append(saved_sample_state[0].assign(tf.zeros([1, self._num_nodes[0]])))
                reset_list.append(saved_sample_state[1].assign(tf.zeros([1, self._num_nodes[0]])))
                reset_list.append(saved_sample_trigger.assign(tf.zeros([1, 1])))

                """reset sample state"""
                self._reset_sample_state = tf.group(*reset_list)
                
                """trigger"""
                sample_output, sample_state, self.local_trigger, trigger, self.trigger_for_plot = self.iteration(self._sample_input,
                                                                                                                 saved_sample_state,
                                                                                                                 saved_sample_trigger)

                sample_save_list = list()
                sample_save_list.append(saved_sample_state[0].assign(sample_state[0]))
                sample_save_list.append(saved_sample_state[1].assign(sample_state[1]))
                sample_save_list.append(saved_sample_trigger.assign(trigger))

                with tf.control_dependencies(sample_save_list):
                    """sample prediction"""
                    self._sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, weights, bias)) 
                
                
                """saver"""
                self.saver = tf.train.Saver(max_to_keep=None)
                            
                        
    
    def _generate_metadata(self, half_life, decay, num_averaging_iterations, optional_feed_dict):
        metadata = list()
        metadata.append(self._batch_size)
        metadata.append(self._num_unrollings)
        metadata.append(self._num_layers)
        metadata.append(list(self._num_nodes))
        metadata.append(half_life)
        metadata.append(decay)
        metadata.append(self._last_num_steps)
        metadata.append(num_averaging_iterations)
        metadata.append(self._init_bias)
        metadata.append(dict(self._threshold))
        metadata.append(optional_feed_dict['self.memory_fine'])
        metadata.append(self._mean)
        metadata.append(self._stddev)
        metadata.append(self._shift)
        metadata.append(self._init_learning_rate)
        metadata.append('adaptive_sum')
        return metadata
  
    def get_triggers(self, session, num_strings=10, length=75, start_positions=None):
        self._reset_sample_state.run()
        self._valid_batches = BatchGenerator(self._valid_text,
                                             1,
                                             self._vocabulary_size,
                                             self._characters_positions_in_vocabulary,
                                             1)
        if start_positions is None:
            start_positions = list()
            if self._valid_size / num_strings < length:
                num_strings = self._valid_size / length
            for i in range(num_strings):
                start_positions.append(i* (self._valid_size / num_strings) + self._valid_size / num_strings / 2)
            while self._valid_size - start_positions[-1] < length:
                del start_positions[-1]
        text_list = list()
        trigger_list = list()
        local_trigger_list = list()
        collect_triggers = False
        letters_parsed = -1
        for idx in range(self._valid_size):
            b = self._valid_batches.next()
            
            if idx in start_positions or collect_triggers: 
                if letters_parsed == -1:
                    letters_parsed = 0
                    text = u""
                    t_list = list()
                    l_t_list = list()
                    collect_triggers = True
                text += characters(b[0], self._vocabulary)[0]
                t_list.append(self.trigger_for_plot.eval({self._sample_input: b[0]}))
                l_t_list.append(self.local_trigger.eval({self._sample_input: b[0]}))
                letters_parsed += 1
                if letters_parsed >= length:
                    collect_triggers = False
                    trigger_list.append(t_list)
                    local_trigger_list.append(l_t_list)
                    text_list.append(text)
                    letters_parsed = -1
                    
            _ = self._sample_prediction.eval({self._sample_input: b[0]})
        return text_list, trigger_list, local_trigger_list 

In [31]:
threshold = {'fixed': True, 'min': 0.5, 'max': 0.7, 'epochs': 10000}

model = adaptive_sum_with_memory(64,
                 vocabulary,
                 characters_positions_in_vocabulary,
                 30,
                 1,
                 [128],
                 0.,
                 threshold,    #{'fixed': True/False, 'min':  , 'max':  ,'epochs':  }
                 train_text,
                 valid_text)

In [17]:
optional_feed_dict = {'self.memory_fine': 0.000001}
model.run(30,
          0.9,
            200,
            50,
            3,
            1,
            20,
          optional_feed_dict=optional_feed_dict,
            print_intermediate_results = True)

Initialized
Average loss at step 0: 5.279430 learning rate: 0.500000
Percentage_of correct: 0.00%

random:

Ú¯¤À+zí
'·Û©»¬tpXµ.SX6o®¬e§wÃô@ûªúõ©W×"VÎ¨F«:p£B£ÿu©.Ü«ÞÚ#Uþ øèÔI¿bùób%»à'h!vÂÎ
H	ÒÐÂ`tþ}¼ºï!t².ÚhÑHÊIqydB`ËËÃÈ<@mwK®dP_¹O(kôU(*ÀÚzYý³K=Úë!Ú­ZÚ)Ò×[NN¸oÃ:þ²ãAw
8i]bé%4ïö4 [O¼´l3H¾'¼ä%ª©B=ÞHÉC°CÔ½WÆ,Çí)å÷Êd(j§Å¸ÛV÷Cò*´eRsô4S¨,7)£Õ6ÏûfíÁMMn~
3ðøt¤I2V~Æð·úuµ(
9<£c\ôöT:ÜkäöRû Éòô_T$+¬{Æ½|Û¨Õz>}©yéHp:kgÀ¼é qÂûÖÆV#ôN#Õ®}5!»
¯bÓ§ÐÒ:vkLll-øxí'Û\7BëPýnèúûi¿ç3¯ùO+Cbck!Í©!´+ý¥<,RÄÌêdµµ$Lï´ÊSâËæUl.
üÖN¯yôf®


KeyboardInterrupt: 

In [9]:
optional_feed_dict = {'self.memory_fine': 0.0000001}
model.simple_run(100,
                       'adaptive_sum_with_memory/test',
                       10000,
                       400,
                       5000,        #learning has a chance to be stopped after every block of steps
                       30,
                       0.9,
                       3,
                       optional_feed_dict=optional_feed_dict,
                        fixed_num_steps=True)

Number of steps = 10000     Percentage = 41.85%     Time = 618s     Learning rate = 0.0212


In [10]:
text_list, trigger_list, local_trigger_list = model.run_for_analitics(model.get_triggers,
                                                        'adaptive_sum_with_memory/test',
                                                        [300, 75, None])
triggers = list()
local_triggers = list()
for text_number, text in enumerate(text_list):
    trig = list()
    l_trig = list()
    text_triggers = trigger_list[text_number]
    text_local_triggers = local_trigger_list[text_number]
    for text_trigger, text_local_trigger in zip(text_triggers, text_local_triggers):
        trig.append(text_trigger[0, 0])
        l_trig.append(text_local_trigger[0, 0])
    triggers.append(trig)
    local_triggers.append(l_trig)
structure_vocabulary_plots(text_list,
                            triggers,
                            'triggers',
                            'mean trigger',
                            ['adaptive_sum_with_memory', 'test_folder'],
                            'triggers',
                            ylims=[0., 1.],
                            ylims_fixed=True,
                            threshold=.5,
                            show=False)
structure_vocabulary_plots(text_list,
                            local_triggers,
                            'local_triggers',
                            'mean trigger',
                            ['adaptive_sum_with_memory', 'test_folder'],
                            'local_triggers',
                            ylims=[0., 1.],
                            ylims_fixed=True,
                            threshold=.5,
                            show=False)
for i in range(99):
    text_plot(text_list[i],
                      triggers[i],
                      'trigger',
                      'triggers',
                      ['adaptive_sum_with_memory', 'test_folder', 'text_triggers'],
                      'triggers#%s' % i,
                      threshold=.5,
                      show=False, 
              sum_flag=True)

    text_plot(text_list[i],
                      local_triggers[i],
                      'local_trigger',
                      'local_triggers',
                      ['adaptive_sum_with_memory', 'test_folder', 'text_local_triggers'],
                      'local_triggers#%s' % i,
                      threshold=.5,
                      show=False, 
              sum_flag=True)

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:1288: UserWarning: findfont: Font family [u'normal'] not found. Falling back to Bitstream Vera Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [7]:
#.001, .01, .1, .2, .3, .5, .7,
threshold_values = [.9]
memory_fine_values = [.00001, .000001, .0000001, .00000001, .0]
init_bias_values = [0., .5, 1., 2., 5.]
threshold = {'fixed': True, 'min': 0.2, 'max': 0.7, 'epochs': 10000}
optional_feed_dict = {'self.memory_fine': 0.001}
num_iter = 20
results_GL = list()
for threshold_value in threshold_values:
    print(' '*4, "threshold: ", threshold_value)
    threshold['min'] = threshold_value
    for memory_fine_value in memory_fine_values:
        print(' '*2, "memory fine: ", memory_fine_value)
        optional_feed_dict['self.memory_fine'] = memory_fine_value
        for init_bias_value in init_bias_values:
            print("init bias: ", init_bias_value)
            model = adaptive_sum_with_memory(64,
                             vocabulary,
                             characters_positions_in_vocabulary,
                             30,
                             1,
                             [128],
                             init_bias_value,
                             threshold,    #{'fixed': True/False, 'min':  , 'max':  ,'epochs':  }
                             train_text,
                             valid_text)
            model.simple_run(100,
                           'adaptive_sum_with_memory/variables/1_128_nu30_ns40k_dcs30/th%s_mf%s_ib%s' % (threshold_value, memory_fine_value, init_bias_value),
                               40000,
                               4000,
                               5000,        #learning has a chance to be stopped after every block of steps
                               30,
                               0.9,
                               3,
                               optional_feed_dict=optional_feed_dict,
                                fixed_num_steps=True)
            results_GL.extend(model._results)
            model.destroy()
            del model
            gc.collect()

     threshold:  0.9
   memory fine:  1e-05
init bias:  0.0
Number of steps = 40000     Percentage = 48.70%     Time = 2455s     Learning rate = 0.0212
init bias:  0.5
Number of steps = 40000     Percentage = 48.49%     Time = 2446s     Learning rate = 0.0212
init bias:  1.0
Number of steps = 40000     Percentage = 48.44%     Time = 2432s     Learning rate = 0.0212
init bias:  2.0
Number of steps = 40000     Percentage = 48.73%     Time = 2417s     Learning rate = 0.0212
init bias:  5.0
Number of steps = 40000     Percentage = 48.55%     Time = 2417s     Learning rate = 0.0212
   memory fine:  1e-06
init bias:  0.0
Number of steps = 40000     Percentage = 48.67%     Time = 2431s     Learning rate = 0.0212
init bias:  0.5
Number of steps = 40000     Percentage = 48.81%     Time = 2419s     Learning rate = 0.0212
init bias:  1.0
Number of steps = 40000     Percentage = 48.82%     Time = 2433s     Learning rate = 0.0212
init bias:  2.0
Number of steps = 40000     Percentage = 48.66%     T

In [15]:
#.001, .01, .1, .2, .3, .5, .7,
threshold_values = [.001, .01, .1, .2, .3, .5, .7, .9]
memory_fine_values = [.00001, .000001, .0000001, .00000001, .0]
init_bias_values = [0., .5, 1., 2., 5.]
threshold = {'fixed': True, 'min': .2, 'max': .7, 'epochs': 10000}
optional_feed_dict = {'self.memory_fine': 0.001}
results_GL = list()
for threshold_value in threshold_values:
    print(' '*4, "threshold: ", threshold_value)
    threshold['min'] = threshold_value
    for memory_fine_value in memory_fine_values:
        print(' '*2, "memory fine: ", memory_fine_value)
        optional_feed_dict['self.memory_fine'] = memory_fine_value
        for init_bias_value in init_bias_values:
            print("init bias: ", init_bias_value)
            model = adaptive_sum_with_memory(64,
                             vocabulary,
                             characters_positions_in_vocabulary,
                             30,
                             1,
                             [128],
                             init_bias_value,
                             threshold,    #{'fixed': True/False, 'min':  , 'max':  ,'epochs':  }
                             train_text,
                             valid_text)
            save_path = 'adaptive_sum_with_memory/variables/1_128_nu30_ns40k_dcs30/th%s_mf%s_ib%s' % (threshold_value, memory_fine_value, init_bias_value)
            model.get_result(save_path,
                             100,
                   40000,
                   30,
                   0.9,
                           optional_feed_dict=optional_feed_dict,  
                        fixed_num_steps=True)
            print('cycle:', optional_feed_dict)
            results_GL.extend(model._results)
            model.destroy()
            del model

     threshold:  0.001
   memory fine:  1e-05
init bias:  0.0
Number of steps = 0     Percentage = 48.63%     Time = 0s     Learning rate = 0.0212
{'self.memory_fine': 1e-05}
init bias:  0.5


KeyboardInterrupt: 

In [14]:
for result in results_GL:
    result['metadata'][model._indices['memory_fine']] = result['metadata'][model._indices['memory_fine']]['self.memory_fine']

In [ ]:
folder_name = 'adaptive_sum_with_memory'
file_name = 'adaptive_sum_with_memory_ns_40000_hl_1333_dc_0.9_ilr1._th0.001-0.9_mf0-1e-5_ib0-5_fixed.pickle'
force = True
pickle_dump = {'results_GL': results_GL}
if not os.path.exists(folder_name):
    try:
        os.makedirs(folder_name)
    except Exception as e:
        print("Unable create folder '%s'" % folder_name, ':', e)    
print('Pickling %s.' % (folder_name + '/' + file_name))
try:
    with open(folder_name + '/' + file_name, 'wb') as f:
        pickle.dump(pickle_dump, f, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save data to', file_name, ':', e)

In [11]:
folder_name = 'adaptive_sum_with_memory'
pickle_file = 'adaptive_sum_with_memory_ns_40000_hl_1333_dc_0.9_ilr1._th0.001-0.9_mf0-1e-5_ib0-5_fixed.pickle'

with open(folder_name + '/' + pickle_file, 'rb') as f:
  save = pickle.load(f)
  results_GL = save['results_GL']
  del save  # hint to help gc free up memory

In [19]:
folder_name = 'adaptive_sum_with_memory'
pickle_file = 'adaptive_sum_with_memory_ns_40000_hl_1333_dc_0.9_ilr1._th0.001-0.9_mf0-1e-5_ib0-5.pickle'

with open(folder_name + '/' + pickle_file, 'rb') as f:
  save = pickle.load(f)
  results_old = save['results_GL']
  del save  # hint to help gc free up memory

In [20]:
# The error occured because of no using constructor for optional_feed_dict
memory_fine_values = [.00001, .000001, .0000001, .00000001, .0]
idx = 0
new_results = list()
for thr_idx in range(8):
    for memory_fine_value in memory_fine_values:
        for other_idx in range(5):
            new_results.append(dict(results_old[idx]))
            new_results[idx]['metadata'][model._indices['memory_fine']] = memory_fine_value
            idx += 1

In [24]:
folder_name = 'adaptive_sum_with_memory'
file_name = 'adaptive_sum_with_memory_ns_40000_hl_1333_dc_0.9_ilr1._th0.001-0.9_mf0-1e-5_ib0-5_fixed.pickle'
force = True
pickle_dump = {'results_GL': new_results}
if not os.path.exists(folder_name):
    try:
        os.makedirs(folder_name)
    except Exception as e:
        print("Unable create folder '%s'" % folder_name, ':', e)    
print('Pickling %s.' % (folder_name + '/' + file_name))
try:
    with open(folder_name + '/' + file_name, 'wb') as f:
        pickle.dump(pickle_dump, f, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save data to', file_name, ':', e)

Pickling adaptive_sum_with_memory/adaptive_sum_with_memory_ns_40000_hl_1333_dc_0.9_ilr1._th0.001-0.9_mf0-1e-5_ib0-5_fixed.pickle.


In [34]:
folder_name = 'adaptive_sum_with_memory'
pickle_file = 'adaptive_sum_with_memory_ns_40000_hl_1333_dc_0.9_ilr1._th0.001-0.9_mf0-1e-5_ib0-5_fixed.pickle'

with open(folder_name + '/' + pickle_file, 'rb') as f:
  save = pickle.load(f)
  results_GL = save['results_GL']
  del save  # hint to help gc free up memory

In [35]:
#.001, .01, .1, .2, .3, .5, .7,
threshold_values = [.001, .01, .1, .2, .3, .5, .7, .9]
memory_fine_values = [.00001, .000001, .0000001, .00000001, .0]
init_bias_values = [0., .5, 1., 2., 5.]
short_results = dict()
for result in results_GL:
    key1 = result['metadata'][model._indices['threshold']]['min']
    key2 = result['metadata'][model._indices['memory_fine']]
    key3 = result['metadata'][model._indices['init_bias']]
    if key1 not in short_results.keys():
        short_results[key1] = dict()
        short_results[key1][key2] = dict()
        short_results[key1][key2][key3] = result['data']['train']['percentage'][-1]
    elif key2 not in short_results[key1].keys():
        short_results[key1][key2] = dict()
        short_results[key1][key2][key3] = result['data']['train']['percentage'][-1]
    elif key3 not in short_results[key1][key2].keys():
        short_results[key1][key2][key3] = result['data']['train']['percentage'][-1]
    else:
        print('Error! Skipping')
        print(result)

In [36]:
print('short_results.keys():', short_results.keys())
print('short_results[first].keys():', short_results[short_results.keys()[0]].keys())
print('short_results[first][first].keys():', short_results[short_results.keys()[0]][short_results[short_results.keys()[0]].keys()[0]].keys())

short_results.keys(): [0.5, 0.1, 0.2, 0.001, 0.3, 0.9, 0.7, 0.01]
short_results[first].keys(): [1e-08, 0.0, 1e-06, 1e-05, 1e-07]
short_results[first][first].keys(): [0.0, 0.5, 2.0, 5.0, 1.0]


In [29]:
#.001, .01, .1, .2, .3, .5, .7,
threshold_values = [.001, .01, .1, .2, .3, .5, .7, .9]
memory_fine_values = [.00001, .000001, .0000001, .00000001, .0]
init_bias_values = [0., .5, 1., 2., 5.]
threshold = {'fixed': True, 'min': .2, 'max': .7, 'epochs': 10000}
optional_feed_dict = {'self.memory_fine': 0.001}
results_GL = list()
for threshold_value in threshold_values:
    print(' '*4, "threshold: ", threshold_value)
    threshold['min'] = threshold_value
    for memory_fine_value in memory_fine_values:
        print(' '*2, "memory fine: ", memory_fine_value)
        optional_feed_dict['self.memory_fine'] = memory_fine_value
        for init_bias_value in init_bias_values:
            print("init bias: ", init_bias_value)
            model = adaptive_sum_with_memory(64,
                             vocabulary,
                             characters_positions_in_vocabulary,
                             30,
                             1,
                             [128],
                             init_bias_value,
                             threshold,    #{'fixed': True/False, 'min':  , 'max':  ,'epochs':  }
                             train_text,
                             valid_text)
            text_list, trigger_list, local_trigger_list = model.run_for_analitics(model.get_triggers,
                                                            'adaptive_sum_with_memory/variables/1_128_nu30_ns40k_dcs30/th%s_mf%s_ib%s' % (threshold_value, memory_fine_value, init_bias_value),
                                                            [300, 75, None])
            
            triggers = list()
            local_triggers = list()
            for text_number, text in enumerate(text_list):
                trig = list()
                l_trig = list()
                text_triggers = trigger_list[text_number]
                text_local_triggers = local_trigger_list[text_number]
                for text_trigger, text_local_trigger in zip(text_triggers, text_local_triggers):
                    trig.append(text_trigger[0, 0])
                    l_trig.append(text_local_trigger[0, 0])
                triggers.append(trig)
                local_triggers.append(l_trig)
                
            structure_vocabulary_plots(text_list,
                                       triggers,
                                       'triggers for letter position (threshold %s, memory fine %s, init_bias %s, result %.2f%%)' % (threshold_value, 
                                                                                                                                     memory_fine_value,
                                                                                                                                     init_bias_value,
                                                                                                                                     short_results[threshold_value][memory_fine_value][init_bias_value]),
                                       'mean trigger',
                                       ['adaptive_sum_with_memory',
                                        'triggers',
                                        'init_bias_%s' % init_bias_value,
                                        'memory_fine_%s' % memory_fine_value,
                                        'threshold_%s_pcnt %.2f' % (threshold_value, short_results[threshold_value][memory_fine_value][init_bias_value])],
                                       'mean_triggers128_ib%s_th%s_mf%s' % (init_bias_value, threshold_value, memory_fine_value),
                                       ylims=[0., 1.],
                                       ylims_fixed=True,
                                       threshold=threshold_value,
                                       show=False)
            structure_vocabulary_plots(text_list,
                                       local_triggers,
                                       'triggers for letter position (threshold %s, memory fine %s, init_bias %s, result %.2f%%)' % (threshold_value, 
                                                                                                                                     memory_fine_value,
                                                                                                                                     init_bias_value,
                                                                                                                                     short_results[threshold_value][memory_fine_value][init_bias_value]),
                                       'mean trigger',
                                       ['adaptive_sum_with_memory',
                                        'local_triggers',
                                        'init_bias_%s' % init_bias_value,
                                        'memory_fine_%s' % memory_fine_value,
                                        'threshold_%s_pcnt %.2f' % (threshold_value, short_results[threshold_value][memory_fine_value][init_bias_value])],
                                       'mean_triggers128_ib%s_th%s_mf%s' % (init_bias_value, threshold_value, memory_fine_value),
                                       ylims=[0., 1.],
                                       ylims_fixed=True,
                                       threshold=threshold_value,
                                       show=False)
            for i in range(50):
                text_plot(text_list[i],
                          triggers[i],
                          'trigger',
                          'mean trigger values (threshold %s, memory fine %s, init_bias %s, result %.2f%%)' % (threshold_value, 
                                                                                                               memory_fine_value,
                                                                                                               init_bias_value,
                                                                                                               short_results[threshold_value][memory_fine_value][init_bias_value]),
                          ['adaptive_sum_with_memory',
                           'triggers',
                           'init_bias_%s' % init_bias_value,
                           'memory_fine_%s' % memory_fine_value,
                           'threshold_%s_pcnt %.2f' % (threshold_value, short_results[threshold_value][memory_fine_value][init_bias_value]),
                           'text_plots'],
                          'triggers128_ib%s_th%s_mf%s#%s' % (init_bias_value, threshold_value, memory_fine_value, i),
                          threshold=threshold_value,
                          show=False,
                          sum_flag=True)
                text_plot(text_list[i],
                          local_triggers[i],
                          'trigger',
                          'mean trigger values (threshold %s, memory fine %s, init_bias %s, result %.2f%%)' % (threshold_value, 
                                                                                                               memory_fine_value,
                                                                                                               init_bias_value,
                                                                                                               short_results[threshold_value][memory_fine_value][init_bias_value]),
                          ['adaptive_sum_with_memory',
                           'local_triggers',
                           'init_bias_%s' % init_bias_value,
                           'memory_fine_%s' % memory_fine_value,
                           'threshold_%s_pcnt %.2f' % (threshold_value, short_results[threshold_value][memory_fine_value][init_bias_value]),
                           'text_plots'],
                          'triggers128_ib%s_th%s_mf%s#%s' % (init_bias_value, threshold_value, memory_fine_value, i),
                          threshold=threshold_value,
                          show=False,
                          sum_flag=False)
            model.destroy()
            del model
            gc.collect() 

     threshold:  0.001
   memory fine:  1e-05
init bias:  0.0


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:1288: UserWarning: findfont: Font family [u'normal'] not found. Falling back to Bitstream Vera Sans
  (prop.get_family(), self.defaultFamily[fontext]))


init bias:  0.5
init bias:  1.0
init bias:  2.0
init bias:  5.0
   memory fine:  1e-06
init bias:  0.0
init bias:  0.5
init bias:  1.0
init bias:  2.0
init bias:  5.0
   memory fine:  1e-07
init bias:  0.0
init bias:  0.5
init bias:  1.0
init bias:  2.0
init bias:  5.0
   memory fine:  1e-08
init bias:  0.0
init bias:  0.5
init bias:  1.0
init bias:  2.0
init bias:  5.0
   memory fine:  0.0
init bias:  0.0
init bias:  0.5
init bias:  1.0
init bias:  2.0
init bias:  5.0
     threshold:  0.01
   memory fine:  1e-05
init bias:  0.0
init bias:  0.5
init bias:  1.0
init bias:  2.0
init bias:  5.0
   memory fine:  1e-06
init bias:  0.0
init bias:  0.5
init bias:  1.0
init bias:  2.0
init bias:  5.0
   memory fine:  1e-07
init bias:  0.0
init bias:  0.5
init bias:  1.0
init bias:  2.0
init bias:  5.0
   memory fine:  1e-08
init bias:  0.0
init bias:  0.5
init bias:  1.0
init bias:  2.0
init bias:  5.0
   memory fine:  0.0
init bias:  0.0
init bias:  0.5
init bias:  1.0
init bias:  2.0
init bi

In [38]:
from plot_module import ComparePlots
plot_options = {'x': 'log'}

adaptive_sum_plots = ComparePlots('adaptive_sum')
adaptive_sum_plots.add_network(results_GL, model._indices)



In [40]:
plot_options={'x': 'log'}
threshold_values = [.001, .01, .1, .2, .3, .5, .7, .9]
memory_fine_values = [.00001, .000001, .0000001, .00000001, .0]
init_bias_values = [0., .5, 1., 2., 5.]
threshold = {'fixed': True, 'min': .2, 'max': .7, 'epochs': 10000}
optional_feed_dict = {'self.memory_fine': 0.001}
results_GL = list()
plot_data, _ = adaptive_sum_plots.one_key_layout_data('adaptive_sum_1',
                                         'memory_fine',
                                         "threshold['min']",
                                                        fixed_variables_list=['init_bias'])
for one_plot_data in plot_data:
    adaptive_sum_plots.save_layout(one_plot_data,
                    'memory fine effect',
                    ['to_move_plots', 'init_bias', 'mf_dep'],
                    'nn128;ns40000;dc0.9;dcs30;ib%s' % one_plot_data['fixed'][('init_bias', None)],
                     plot_options=plot_options)
    
plot_data, _ = adaptive_sum_plots.one_key_layout_data('adaptive_sum_1',
                                         "threshold['min']",
                                                           'memory_fine',
                                                        fixed_variables_list=['init_bias'])
for idx, one_plot_data in enumerate(plot_data):
    adaptive_sum_plots.save_layout(one_plot_data,
                    'threshold effect',
                    ['to_move_plots', 'init_bias', 'thr_dep'],
                    'nn128;ns40000;dc0.9;dcs30;ib%s_log' % one_plot_data['fixed'][('init_bias', None)],
                     plot_options=plot_options)
for idx, one_plot_data in enumerate(plot_data):
    adaptive_sum_plots.save_layout(one_plot_data,
                    'threshold effect',
                    ['to_move_plots', 'init_bias', 'thr_dep'],
                    'nn128;ns40000;dc0.9;dcs30;ib%s' % one_plot_data['fixed'][('init_bias', None)])
    


In [41]:
plot_data, _ = adaptive_sum_plots.one_key_layout_data('adaptive_sum_1',
                                         'memory_fine',
                                         "init_bias",
                                                        fixed_variables_list=["threshold['min']"])
for one_plot_data in plot_data:
    adaptive_sum_plots.save_layout(one_plot_data,
                    'memory fine effect',
                    ['to_move_plots', "threshold['min']", 'mf_dep'],
                    'nn128;ns40000;dc0.9;dcs30;thr%s' % one_plot_data['fixed'][('threshold', 'min')],
                     plot_options=plot_options)
    
plot_data, _ = adaptive_sum_plots.one_key_layout_data('adaptive_sum_1',
                                         "init_bias",
                                         'memory_fine',
                                                        fixed_variables_list=["threshold['min']"])
for one_plot_data in plot_data:
    adaptive_sum_plots.save_layout(one_plot_data,
                    'init_bias effect',
                    ['to_move_plots', "threshold['min']", 'ib_dep'],
                    'nn128;ns40000;dc0.9;dcs30;thr%s' % one_plot_data['fixed'][('threshold', 'min')])

In [42]:
plot_data, _ = adaptive_sum_plots.one_key_layout_data('adaptive_sum_1',
                                         "threshold['min']",
                                         "init_bias",
                                                        fixed_variables_list=['memory_fine'])
for one_plot_data in plot_data:
    adaptive_sum_plots.save_layout(one_plot_data,
                    'threshold effect',
                    ['to_move_plots', "memory_fine", 'thr_dep_log'],
                    'nn128;ns40000;dc0.9;dcs30;mf%s' % one_plot_data['fixed'][('memory_fine', None)],
                     plot_options=plot_options)
for one_plot_data in plot_data:
    adaptive_sum_plots.save_layout(one_plot_data,
                    'threshold effect',
                    ['to_move_plots', "memory_fine", 'thr_dep'],
                    'nn128;ns40000;dc0.9;dcs30;mf%s' % one_plot_data['fixed'][('memory_fine', None)])
    
plot_data, _ = adaptive_sum_plots.one_key_layout_data('adaptive_sum_1',
                                         "init_bias",
                                         "threshold['min']",
                                                        fixed_variables_list=['memory_fine'])
for one_plot_data in plot_data:
    adaptive_sum_plots.save_layout(one_plot_data,
                    'init_bias effect',
                    ['to_move_plots', "memory_fine", 'ib_dep'],
                    'nn128;ns40000;dc0.9;dcs30;mf%s' % one_plot_data['fixed'][('memory_fine', None)])